<a href="https://colab.research.google.com/github/asakata0821/Aisa_INFO5731_-Fall2023/blob/main/Sakata_Aisa_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 reddits by using a hashtag (you can use any hashtag) from Reddits.


In [4]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 1s (227 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22

In [40]:
#I selected (2), collecting review text data from IMDB.
#I chose "Top Gun: Maverick"
#This was filmed in 2022, and it has only 4,261 reviews.
#If we have to choose the new movies (in 2022 or 2023), I think the number of reviews won't reach 10000.


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import csv

all_list = []

#Because this website doesn't iterate by pagenumber in the link, but it shows more and more reviews by clicking the button.
#In this case, we use selenium to click the button to see more information.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'
}

url = 'https://www.imdb.com/title/tt1745960/reviews?sort=submissionDate&dir=desc&ratingFilter=0'

driver.get(url)

while True:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'ipl-load-more__button'))
        )
        load_more_button.click()
    except Exception as e:
        #Till all reviews are shown
        print("No more reviews to load.")
        break

response = driver.page_source

driver.quit()


#webscraping
soup = BeautifulSoup(response, 'html.parser')
result = soup.find_all(class_='review-container')

#Collecting usernames
for item in result:
  namepart = item.find(class_='display-name-link')
  if namepart:
      username = namepart.a.get_text()
      print("Username:", username)
  else:
        print("Username not found in this review.")

#Collecting post date
  datepart = item.find(class_='review-date')
  if datepart:
      date = datepart.get_text()
      print("Review Date:", date)
  else:
      print("Review Date not found in this review.")

#Collecting the header of each review
  titlepart = item.find(class_='title')
  if titlepart:
      title = titlepart.get_text()
      print("Title:", title, end="")
  else:
      print("Title not found in this review.")

#Collecting the review contents
  reviewpart = item.find(class_='text show-more__control')
  if reviewpart:
      review = reviewpart.get_text()
      print("Review:", review)
  else:
      print("Review not found in this review.")
  print()


  all_list.append({
      'Username': username,
      'Review Date': date,
      'Title': title,
      'Review': review
  })



#Title of the csv file
csv_file_path = 'IMDB_reviews.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['Username', 'Review Date', 'Title', 'Review']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for item in all_list:
        writer.writerow({
            'Username': item['Username'],
            'Review Date': item['Review Date'],
            'Title': item['Title'],
            'Review': item['Review']
        })

print('Data saved to', csv_file_path)


Streaming output truncated to the last 5000 lines.
Review Date: 30 May 2022
Title:  Tom cruise can do anything.
Review: This movie is a grand show for all.Tom cruise puts everything that he can do. I love the film so much. Every actor of the movie did great job.Recommended for every one.

Username: johnfanning-53228
Review Date: 30 May 2022
Title:  Awesome!
Review: This is one of the few sequels that exceeds the original. Not taking anything away from the first one, but this one reaches heights no other movie has in recent memory. Better still it is not a comic book movie. Tom Cruise delivers an outstanding performance, Miles Teller looks just like a young Anthony Edwards, and Jennifer Connelly is still smoking hot. If you like the first Top Gun, you will love this one.

Username: bikipramanic
Review Date: 30 May 2022
Title:  Epic
Review: Just watch the flim in a good theatre and feel the movie. Tom's movies need no review , just go and watch............................................

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [50]:
pip install nltk

In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

#When you grade, I assume that you don't use these lines below, but you run the code above.
#I assuem that you'll get all_list when you run all code above

#This line is for when we upload the csv file on this google colab
all_list = pd.read_csv('IMDB_reviews.csv')
df = pd.DataFrame(all_list)
title = df['Title']
review = df['Review']

#For the functions
punctuations = [',', '.', ':', ';', '!', '?', '(', ')', '[', ']', '{', '}', "'", '"', '-', '_', '/', '\\', '|', '+', '=', '*', '&', '^', '%', '$', '#', '@', '<', '>', '~']
nltk.download('stopwords')
nltk.download('wordnet')


def remove_punctuation(text):
    for i in punctuations:
        text = text.replace(i, '')
    return text

def remove_number(text):
    text = ''.join(i for i in text if not i.isdigit())
    return text

def remove_stopwords(text):
    stopword = stopwords.words('english')
    text = ' '.join(i for i in text.split() if i not in stopword)
    return text

def lowercasing(text):
    return text.lower()

def stemming(text):
    stemmer = PorterStemmer()
    return ' '.join(stemmer.stem(i) for i in text.split())

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join(lemmatizer.lemmatize(i) for i in text.split())

# Apply all functions to each column
cleaned_title = title.apply(remove_punctuation).apply(remove_number).apply(remove_stopwords).apply(lowercasing).apply(stemming).apply(lemmatization).str.replace('\n', '')
cleaned_review = review.apply(remove_punctuation).apply(remove_number).apply(remove_stopwords).apply(lowercasing).apply(stemming).apply(lemmatization).str.replace('\n', '')



cleaned_data = pd.DataFrame({
    'Username': df['Username'],
    'Review Date': df['Review Date'],
    'Title': title,
    'Cleaned Title': cleaned_title,
    'Review': review,
    'Cleaned Review': cleaned_review
})

# Concatenate the cleaned data with the original data
all_list = pd.concat([all_list, cleaned_data], ignore_index=True)

# Define the CSV file path
csv_file_path = 'IMDB_cleaned_reviews.csv'

# Write the data to the CSV file
cleaned_data.to_csv(csv_file_path, index=False)

print('Data saved to', csv_file_path)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Data saved to IMDB_cleaned_reviews.csv


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes:

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [56]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter

#When you grade, I assume that you don't use these lines below, but you run the code above.
#I assuem that you'll get all_list when you run all code above

#This line is for when we upload the csv file on this google colab
cleaned_df = pd.read_csv('IMDB_cleaned_reviews.csv')
cleaned_title = cleaned_df['Cleaned Title']
cleaned_review = cleaned_df['Cleaned Review']

#For the function
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


def postagging(text):
    if isinstance(text, str) and text.strip():
        # Tokenize the input texts
        words = nltk.word_tokenize(text)
        pos_tags = pos_tag(words)
        # Counting each POS tag (N(oun), V(erb), Adj(ective), Adv(erb))
        pos_tag_counts = Counter(tag for word, tag in pos_tags)

    else:
        #error handling
        pos_tag_counts = Counter()

    return pos_tag_counts

#Apply the function to each, title and review
counts_title = cleaned_title.apply(postagging)
counts_review = cleaned_review.apply(postagging)

# Calculating each POS tag (N(oun), V(erb), Adj(ective), Adv(erb))
total_nouns_title = counts_title.apply(lambda x: x.get('NN', 0) + x.get('NNS', 0))
total_verbs_title = counts_title.apply(lambda x: x.get('VB', 0) + x.get('VBD', 0) + x.get('VBG', 0) + x.get('VBN', 0) + x.get('VBP', 0) + x.get('VBZ', 0))
total_adjectives_title = counts_title.apply(lambda x: x.get('JJ', 0) + x.get('JJR', 0) + x.get('JJS', 0))
total_adverbs_title = counts_title.apply(lambda x: x.get('RB', 0) + x.get('RBR', 0) + x.get('RBS', 0))

total_nouns_review = counts_review.apply(lambda x: x.get('NN', 0) + x.get('NNS', 0))
total_verbs_review = counts_review.apply(lambda x: x.get('VB', 0) + x.get('VBD', 0) + x.get('VBG', 0) + x.get('VBN', 0) + x.get('VBP', 0) + x.get('VBZ', 0))
total_adjectives_review = counts_review.apply(lambda x: x.get('JJ', 0) + x.get('JJR', 0) + x.get('JJS', 0))
total_adverbs_review = counts_review.apply(lambda x: x.get('RB', 0) + x.get('RBR', 0) + x.get('RBS', 0))


print('Title:')
print("Total Nouns in Title:", total_nouns_title.sum())
print("Total Verbs in Title:", total_verbs_title.sum())
print("Total Adjectives in Title:", total_adjectives_title.sum())
print("Total Adverbs in Title:", total_adverbs_title.sum())
print()
print('Review:')
print("Total Nouns in Review:", total_nouns_review.sum())
print("Total Verbs in Review:", total_verbs_review.sum())
print("Total Adjectives in Review:", total_adjectives_review.sum())
print("Total Adverbs in Review:", total_adverbs_review.sum())







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Title:
Total Nouns in Title: 8787
Total Verbs in Title: 1559
Total Adjectives in Title: 2820
Total Adverbs in Title: 987

Review:
Total Nouns in Review: 123758
Total Verbs in Review: 36857
Total Adjectives in Review: 48718
Total Adverbs in Review: 13515


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):**aa

Constituency parsing tree:
Constituency parsing is a process of analyzing sentences by breaking them down into sub-phrases, such as nouns, verbs, adjectives, and adverbs. We can create a tree with this structure, and each branch represents a group of words that are meaningful to all. This helps us to see which words are relevant in a sentence.

Dependency parsing tree:
Dependency parsing is a method to analyze the grammatical structure in a sentence by extracting a dependency parse of a sentence. It defines the relationships between words. The structure looks like a map, and the lines between them show how they are related grammarly. These lines have labels, which show us the relationship of words, such as subject, object, and modifier.